### Collect final targets, notes, overrides, crepe-assisted estimates and ground estimates 
### for the three parts of all songs

Graph them all. 

Compare targets to ground estimates.

Compare crepe-assisted estimates to ground estimates.

In [ ]:
import numpy as np
import librosa
import matplotlib.pyplot as plt
from scipy.signal import butter
from scipy.fft import fft, ifft, rfft, irfft
import random
from scipy.signal import argrelmin, argrelmax, argrelextrema
import os

In [ ]:
import plotly.graph_objs as go
import plotly.offline as py
from plotly.subplots import make_subplots
import pandas as pd
import plotly.io as pio
from plotly.offline import init_notebook_mode, iplot, plot
from IPython.display import display, clear_output
from ipywidgets import widgets, Button, HBox, VBox
from plotly.colors import DEFAULT_PLOTLY_COLORS
from plotly.callbacks import LassoSelector, BoxSelector


In [ ]:
raw_dir = "/Akamai/voice/data/"
estimate_dir = "/Akamai/voice/data/ground-estimate/"
collections = {"sm":"Scherbaum Mshavanadze",
               "guria":"Teach Yourself Gurian Songs",
               "megrelia":"Teach Yourself Megrelian Songs"}

collection_directories = {"sm":
                          ["GVM009_BatonebisNanina_Tbilisi_Mzetamze_20160919",
                           "GVM017_ChvenMshvidobaTake2_Ozurgeti_ShalvaChemo2016_20160713",
                           "GVM019_DaleKojas_DidgoriVillage_Didgori_20160707",
                           "GVM031_EliaLrde_LakhushdiVillage_MuradGigoGivi_20160819",
                           "GVM097_KristeAghsdga_LakhushdiVillage_MuradGigoGivi_20160819"],
                          "guria":
                          ["Adila-Alipasha",
                           "Alaverdi",
                           "Beri Ak'vans Epareba", 
                           "Brevalo",             
                           "Chven-Mshvidoba",    
                           #'Didi Khnidan',     
                           "Gakhsovs, T'urpa",
                           "Indi-Mindi",
                           "K'alos Khelkhvavi",
                           "Khasanbegura",     
                           "Lat'aris Simghera",    
                           "Manana",         
                           "Maq'ruli",               
                           "Masp'indzelsa Mkhiarulsa", 
                           "Me-Rustveli",        
                           'Mival Guriashi (1)' ,
                           'Mival Guriashi (2)' , 
                           "Mok'le Mravalzhamieri",
                           "Mts'vanesa Da Ukudosa", 
                           'Nanina (1)',      
                           'Nanina (2)',          
                           "Orira",
                           "P'at'ara Saq'varelo",                
                           'Pikris Simghera',
                           "Sabodisho",
                           'Sadats Vshobilvar',
                           "Shermanduli",
                           "Shvidk'atsa",
                           'Supris Khelkhvavi',
                           "Ts'amok'ruli"],
                         "megrelia":
                          ["Vojanudi Chkim Jargvals",
                           "Ak'a Si Rekisho",
                           "Gepshvat Ghvini",
                           "Io _ Chkin Kiana",
                           "Mesishi Vardi",
                           "Meureme",
                           "Mi Re Sotsodali_",
                           "Mole Chit'i Gilakhe",
                           "O Da"]}

ground_estimate_dir = "/Akamai/voice/data/ground-estimate/"
target_dir = "/Akamai/voice/data/pitches-postprocessed/crepe/"
overrides_dir = "/Akamai/voice/data/pitch-overrides/crepe/"
crepe_assisted_dir = "/Akamai/voice/data/pitches-vuv-crepe-assisted/"

In [ ]:
# Convert override box [x1 y1 x2 y2] to x and y traces
def boxesToTrace(boxLines):
    newX = np.zeros(2*len(boxLines))
    newY = np.zeros(2*len(boxLines))
    lineno = 0
    for line in boxLines:
        lineArray = line.split(" ")
        [lineLeftX, lineLeftY, lineRightX, lineRightY] = [float(numeric_string) for numeric_string in lineArray]
        newX[2*lineno:2*lineno+2] = [lineLeftX, lineRightX]
        newY[2*lineno:2*lineno+2] = [lineLeftY, lineRightY]
        lineno += 1
    return (newX, newY)

def get_boxes(file):
    try:
        #print(f"Getting box overrides from {file}")
        f = open(file)
    except (IOError, FileNotFoundError):
        return (np.empty(shape=(0)), np.empty(shape=(0)))
    else:
        with f:
            return boxesToTrace(f.read().splitlines())


In [ ]:
data = {}
locations = {}
algos = ['boersma', 'crepe', 'hermes', 'noll', 'yin']
working_song = "Vojanudi Chkim Jargvals"

def separate(adir):
    conv={}
    conv[0] = lambda s: float(s.strip() or 0)
    x,y = np.loadtxt(adir, unpack=True, usecols=(0,1), converters=conv)
    return (x,y)
        
def load_songs():
    for coll, collection in collections.items():
        print(" ", collection)
        for song in collection_directories[coll]:
            if song != working_song:
                continue
            songDir = song
            if coll == 'sm':
                songDir = song[:6]
            data[songDir] = {}
            print("  ", song)
            for part in ['1', '2', '3']:
                shiftedPartFile = 'AHDS' + part + 'M_shifted.txt'
                shiftedNoteFile = 'AHDS' + part + 'M.notes_shifted.txt'
                boxes_file = f"{overrides_dir}{collection}/{song}/{song}_{shiftedPartFile}"                        
                boxes = get_boxes(boxes_file)
                x, y = separate(f"{target_dir}{collection}/{song}/{song}_{shiftedPartFile}")
                xn, yn = separate(f"{target_dir}{collection}/{song}/{song}_{shiftedNoteFile}")
                xg, yg = separate(f"{ground_estimate_dir}{collection}/{songDir}/{shiftedPartFile}")
                algoPitches = []
                for algo in algos:
                    xa, ya = separate(f"{crepe_assisted_dir}{algo}/{collection}/{song}/{song}_{shiftedPartFile}")
                    algoPitches.append(ya)
                algoPitches = tuple(algoPitches)
                data[songDir][part] = (x, y, yn, yg, boxes, algoPitches)
    print("\nLoaded song data from files into dictionary")
                       
load_songs()                                
                                

### Ground estimate, target, notes, and overrides for all parts on one graph


In [ ]:
def graph_target_estimate(song, reload=False):
    if reload:
        load_songs()
    print("plotting data from dictionary")
    traces = []
    traceId = 0
    for part, res in data[song].items():
        try:
            # target
            trace = go.Scattergl(
                        x = res[0],
                        y = res[1],
                        name="Part " + part + " target",
                        mode="markers",
                        visible=True
                    )

            traceId += 1
            traces.append(trace)
            
            # target boxes
            trace = go.Scattergl(
                        x = res[4][0],
                        y = res[4][1],
                        name="Part " + part + " boxes",
                        mode="markers",
                        marker = dict( symbol="cross-thin",
                                       line=dict(color="black", width=1)
                                     ),
                        visible=True
                    )

            traceId += 1
            traces.append(trace)
            
            # final estimate
            trace = go.Scattergl(
                        x = res[0],
                        y = res[3],
                        name="Part " + part + " estimate",
                        mode="markers",
                        visible=True
                    )

            traceId += 1
            traces.append(trace)
            
            # target notes
            trace = go.Scattergl(
                        x = res[0],
                        y = res[2],
                        name="Part " + part + " notes",
                        mode="markers",
                        visible=True
                    )

            traceId += 1
            traces.append(trace)
            
        except:
            print(f"Part {part} not available")

    layout = go.Layout(title='Activity Heatmap')

    figure = go.Figure(data=traces, layout=layout)

    fig = go.FigureWidget(figure)

    fig.update_layout(
        legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
        ),
        margin=dict(l=0, r=0, t=100, b=0)
    )


    fig.update_traces(
        marker=dict(size=3),
        selector=dict(mode='markers')
    )


    fig['layout']['title'] = "Part estimates for " + song
    fig['layout']['width'] = 900
    fig['layout']['height'] = 500
    fig['layout']['showlegend'] = True

    display(widgets.VBox([fig]))
   

### Ground estimate, target, notes, overrides, crepe-assisted estimate for each part on one graph


In [ ]:
def graph_estimates(song, reload=False):
    if reload:
        load_songs()
    print("plotting data from dictionary")
    traces = []
    type_list = []
    traceId = 0
    for part, res in data[song].items():
        try:
            # target
            trace = go.Scattergl(
                        x = res[0],
                        y = res[1],
                        name="Part " + part + " target",
                        mode="markers",
                        visible= (True if part == "1" else False)
                    )

            traceId += 1
            traces.append(trace)
            type_list.append(part)
            
            # target boxes
            trace = go.Scattergl(
                        x = res[4][0],
                        y = res[4][1],
                        name="Part " + part + " boxes",
                        mode="markers",
                        marker = dict( symbol="cross-thin",
                                       line=dict(color="black", width=1)
                                     ),
                        visible= (True if part == "1" else False)
                    )

            traceId += 1
            traces.append(trace)
            type_list.append(part)
            
            # final estimate
            trace = go.Scattergl(
                        x = res[0],
                        y = res[3],
                        name="Part " + part + " estimate",
                        mode="markers",
                        visible= (True if part == "1" else False)
                    )

            traceId += 1
            traces.append(trace)
            type_list.append(part)
            
            # target notes
            trace = go.Scattergl(
                        x = res[0],
                        y = res[2],
                        name="Part " + part + " notes",
                        mode="markers",
                        visible= (True if part == "1" else False)
                    )

            traceId += 1
            traces.append(trace)
            type_list.append(part)
            
            # algorithm pitches
            for idx in range(len(algos)):
                trace = go.Scattergl(
                            x = res[0],
                            y = res[5][idx],
                            name="Part " + part + " " + algos[idx],
                            mode="markers",
                            visible= (True if part == "1" else False)
                        )

                traceId += 1
                traces.append(trace)
                type_list.append(part)
            
        except:
            print(f"Part {part} not available")

    layout = go.Layout(title='Activity Heatmap')

    figure = go.Figure(data=traces, layout=layout)

    fig = go.FigureWidget(figure)

    buttons = []
    labels = {'1':'bass', '2':'middle', '3':'top'} #, 'mix']
    for part, label in labels.items():
        visibility = [part==current_type for current_type in type_list]
        button = dict(
            label = label,
            method = "update",
            args = [{ 'visible': visibility}]
        )

        buttons.append(button)

    updatemenus = list([
        dict(active=0,
            buttons=buttons,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            xanchor="left",
            yanchor="top",
            x = 0.005,
            y = 1.06,
        )
    ])

    fig.update_layout(
        legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
        ),
        margin=dict(l=0, r=0, t=100, b=0)
    )


    fig.update_traces(
        marker=dict(size=3),
        selector=dict(mode='markers')
    )


    fig['layout']['title'] = "Part estimates for " + song
    fig['layout']['width'] = 900
    fig['layout']['height'] = 500
    fig['layout']['showlegend'] = True
    fig['layout']['updatemenus'] = updatemenus

    display(widgets.VBox([fig]))
   

In [ ]:
graph_target_estimate(working_song, False)

In [ ]:
graph_estimates(working_song, False)

### Notes on ground estimates

#### Alaverdi
- 71.9-72.1sec, top: algorithms disagree up to 10%

#### Brevalo
- 24-26sec, middle: algorithms disagree up to 10%, chord: 158- fixed
- 40.7-41.7sec, top: algorithms disagree up to 10%, chord: 185 - fixed

#### Chven Mshvidoba
- 25.8-26.9sec, top: algorithms disagree up to 7%, chord: 153 
- 78.9-79.9sec, top: algorithms disagree up to 7%, chords: 142, 131
- 86.5-86.6sec, top: algorithms disagree up to 7%
- 172.7-172.9sec, top: algorithms disagree up to 7%, chord: 131

#### K'alos Khelkhvavi
- 106.1-106.3, middle: algorithms disagree up to 10%, chord: 143
- top: some consonant scoops unison with bass, middle

#### Khasanbegura
- on refrain, bass choir, middle choir?
- bass: some ranges fail to isolate correct note for any algorithm


#### Lat'aris Simghera
- 153sec, bass: algorithms disagree up to 7%, chord: 133
- 209.5sec, bass: algorithms disagree up to 9%, chord: 130
- 67.4-67.5sec, middle: algorithms disagree up to 5%, chord: ~154
- 77.8sec, top: algos outside range: Hermes
- 92.8-94.5sec, top: algos outside range
- 95.1sec, top: algos outside range
- 182-187sec, top: algos outside range: Yin, Hermes

#### Manana
- 8sec, middle: delete
- 44-47sec, middle: algorithms disagree up to 15%, first chord 158, fixed
- 65-68sec, middle: algorithms disagree up to 15%, first chord 158, fixed
- 94-95sec, middle: algorithms disagree up to 10%, chord 158, fixed

#### Maq'ruli
- one bass choir
- 19.6-19.9, middle: algorithms disagree up to 15% ... and other places
- 107.7, middle: algos outside range: Yin, Hermes ... and other places
- top: similar problems

#### Mival Guriashi (2)
- 100.7, middle: algorithms disagree up to 10% ... and 104.6

#### Mok'le Mravalzhamieri
- 20.5-20.9, top: algorithms disagree up to 10%, and algos outside range: Hermes

#### Mts'vanesa Da Ukudosa
- 21.6-21.7, middle: algorithms disagree up to 7%

#### Nanina (1)
- 48.9-52.7, top: algorithms disagree up to 10%
- 56-61, top: algorithms disagree up to 10%, and algos outside range: Hermes... and other places, fixed many
- 59.07, top: note = -1 when it should be 0

#### Nanina (2)
- shift may not be perfect: middle ahead at start of song

#### Orira
- one bass choir, not as accurate: not fixing
- 11.4, middle: algorithms disagree up to 10% ... and other places
- 95.6, middle: algos outside range: Hermes
- 160-170, middle: unsalvageable notes 
- top: not fixing inaccuracies in krimanchuli

#### Sabodisho
- chonguri
- middle: algorithms disagree on 158 chords, e.g. 122, and 135 chords, e.g. 194

#### Shvidk'atsa
- one bass choir, not as accurate: not fixing
- 20.9-21.7, middle: algos outside range: Hermes, Yin
- 94-95, middle: algorithms disagree up to 10%, fixed

#### Supris Khelkhvavi
- shift may not be perfect: middle ahead at start of song

#### Ts'amok'ruli
- shift is good from 0 to 46-ish; bad after that

#### Vojanudi Chkim Jargvals
- 117, middle, algorithms disagree up to 10%, fixed
- 10-11, top, algorithms disagree up to 10%, fixed
- 30-35, top, algorithms disagree and algos out of range: Yin
- 52-56, top, algorithms disagree and algos out of range: Yin, Hermes
- 59-60, top, algorithms disagree up to 10%, fixed
- 72-73, top, algorithms disagree up to 10%, fixed
- 91, top, algorithms disagree
- 100-109, top, algorithms disagree and algos out of range: Yin, Hermes, fixed 108
- 112-124, top, algorithms disagree and algos out of range: Yin, Hermes, fixed 123

#### Ak'a Si Rekisho
- bass: Hermes out of range throughout, Yin sometimes; problems at: 51, 78-80, 104-106, 134-138, 143-144, 173-174
- (some of these places are voiced consonants)
- top: Hermes out of range throughout, Yin sometimes; problems at: 49-50, 72-74, 78-79, 165, 172

#### Io _ Chkin Kiana
- 22, top: algos out of range: Yin, Hermes, ... other places: 38-41, 49-51, 66-80, 88-104

#### Mesishi Vardi
- top: algorithms disagree some throughout

#### Meureme
- one top estimate over 6000Hz
- 157.6, top: algos out of range: Yin

#### Mi Re Sotsodali_
- chonguri
- one bass estimate over 2000Hz
- 18.6-18.7, top: algos out of range: Yin
- 38.2-38.7, top: algorithms disagree within range, narrowed
- 39.6-40.3, top: algorithms disagree within range
- 75.7-77.5, top: algorithms disagree within range, narrowed
- 91.49-92.3, top: algorithms disagree, boersma 0 gets vote
- 100.6, 101.5-103.3, top: algos out of range: Yin, fixed/narrowed
- 108-108.6, top: algos out of range: Yin, Boersma 0
- 123.9-125.3, top: algos out of range: Yin, Boersma 0, Hermes at boundary, narrowed
- 138.1-140.1, top: algos out of range: Yin, Boersma 0, Hermes at boundary, narrowed
- 164.9-165.7, top: algos out of range: Yin, Boersma 0, Hermes at boundary, narrowed

#### O Da
- 77.1-77.8, middle: algorithms disagree up to 7%
